In [1]:
# Example file names, adjust appropriately
price_file='TQQQ_yahoo.csv'
split_file='TQQQ_splits.csv'

In [156]:
import numpy as np
import pandas as pd
import os
# load daily yahoo price data and split history
yahoo_target=pd.read_csv(price_file)
yahoo_splits=pd.read_csv(split_file)
# scale daily price data back to original based on split history
yahoo_target['Date']=pd.to_datetime(yahoo_target['Date'])
yahoo_splits['Date']=pd.to_datetime(yahoo_splits['Date'])
multiples=np.array([1])
if len(yahoo_splits)>0:
    yahoo_splits=yahoo_splits.sort_values('Date',ascending=False)
    def multiplier(x):
        tmp=x.split(':')
        return int(tmp[0])/int(tmp[1])
    multiples=np.concatenate((multiples,yahoo_splits['Stock Splits'].apply(multiplier).values))
    factor=1
    for i in range(len(multiples)):
        multiples[i]*=factor
        factor=multiples[i]
split_dates=yahoo_splits['Date'].values
def scale_back(date):
    for i in range(len(split_dates)):
        if date>split_dates[i]:
            return multiples[i]
    return multiples[-1]
v_scale_back=np.vectorize(scale_back)
scales=v_scale_back(yahoo_target['Date'])
for c in ['Open','High','Low','Close','Adj Close']:
    yahoo_target[c]=yahoo_target[c]*scales
# scale all prices
def time_transform(t):
    dt=t.split(' ')[0].split('.')
    return dt[2]+'-'+dt[1]+'-'+dt[0]
files=os.listdir(price_file.split('_')[0])
for i in range(len(files)):
    print('                                        ',end='\r')
    print(i,'/',len(files),end='\r')
    dat=pd.read_csv(price_file.split('_')[0]+"//"+files[i])
    tmp=np.asarray(dat['Ask'].astype(float)+dat['Bid'].astype(float))/2
    OHLC=np.asarray([tmp[0],np.max(tmp),np.min(tmp),tmp[-1]])
    target_OHLC=yahoo_target.iloc[np.where(yahoo_target['Date']==pd.to_datetime(dat['Local time'].apply(time_transform)).values[0])[0][0]].values[1:5].astype(float)
    m,b = np.polyfit(OHLC, target_OHLC, 1)
    dat['Ask']=np.round(dat['Ask'].astype(float)*m+b,2)
    dat['Bid']=np.round(dat['Bid'].astype(float)*m+b,2)
    dat.to_csv(price_file.split('_')[0]+"//"+files[i],index=False)